In [1]:
import sys
sys.path.append("/home/rchoudhu/research/voxelpose-pytorch/lib")

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "7"
import numpy as np
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.utils.data
import torchvision.transforms as transforms
from tensorboardX import SummaryWriter
import argparse
import pprint
import logging
import json
import time
%matplotlib agg
import matplotlib.pyplot as plt

from tqdm import tqdm

#import _init_paths
from core.config import config
from core.config import update_config
from core.function import train_3d, validate_3d
from utils.utils import create_logger
from utils.utils import save_checkpoint, load_checkpoint, load_model_state
from utils.utils import load_backbone_panoptic
from utils.vis import save_debug_3d_images
import dataset
import models

viz_test_dir = "video_viz"

updated.


In [2]:
cfg = "configs/campus/prn64_cpn80x80x20_cam2.yaml"
update_config(cfg)
logger, final_output_dir, tb_log_dir = create_logger(config, cfg, 'validate')

gpus = [int(i) for i in config.GPUS.split(',')]
print('=> Loading data ..')
normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
print(config.DATASET.TEST_SUBSET)
test_dataset = eval('dataset.' + config.DATASET.TEST_DATASET)(
    config, config.DATASET.TEST_SUBSET, False,
    transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

print(test_dataset.image_set)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=config.TEST.BATCH_SIZE * len(gpus),
    shuffle=False,
    num_workers=config.WORKERS,
        pin_memory=True)

print('=> Done loading data.')

=> load /home/rchoudhu/research/voxelpose-pytorch/data/CampusSeq1/pred_campus_maskrcnn_hrnet_coco.pkl


=> creating /home/rchoudhu/research/voxelpose-pytorch/output/campus_synthetic/multi_person_posenet_50/prn64_cpn80x80x20_cam2
=> creating /home/rchoudhu/research/voxelpose-pytorch/log/campus_synthetic/multi_person_posenet_50/prn64_cpn80x80x20_cam22022-02-08-17-52
=> Loading data ..
validation


/home/rchoudhu/research/voxelpose-pytorch/lib/dataset/campus.py:98: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  actor_3d = np.array(np.array(data['actor3D'].tolist()).tolist()).squeeze()  # num_person * num_frame


validation
=> Done loading data.


In [3]:
print('=> Constructing models ..')
# Set is_train to false
model = eval('models.' + config.MODEL + '.get_multi_person_pose_net')(
    config, is_train=False)
print("Done constructing models.")

gpus=[0]
print("Setting data parallel with gpus: " + str(gpus))
start_time = time.time()
with torch.no_grad():
    model = torch.nn.DataParallel(model, device_ids=gpus).cuda()
print("Took %.3f to set up data parallel" % (time.time() - start_time))

=> Constructing models ..
Done constructing models.
Setting data parallel with gpus: [0]
Took 4.656 to set up data parallel


In [4]:
test_model_file = os.path.join(final_output_dir, config.TEST.MODEL_FILE)
if config.TEST.MODEL_FILE and os.path.isfile(test_model_file):
    logger.info('=> load models state {}'.format(test_model_file))
    model.module.load_state_dict(torch.load(test_model_file))
else:
    raise ValueError('Check the model file for testing!')

#print("Uncomment to actually run the validation, but won't tell us anything we don't know yet")
# Add viz stuff to Panoptic. 
# Test the metrics resulting from the different thresholds.
validate_3d(config, model, test_loader, final_output_dir)

=> load models state /home/rchoudhu/research/voxelpose-pytorch/output/campus_synthetic/multi_person_posenet_50/prn64_cpn80x80x20_cam2/model_best.pth.tar
/home/rchoudhu/.conda/envs/voxelpose_test/lib/python3.9/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1640811803361/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/home/rchoudhu/research/voxelpose-pytorch/lib/core/proposal.py:19: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  indices_x = (i

NUM PERSON : 3




     | Actor 1 | Actor 2 | Actor 3 | Average | 
 PCP |  96.12  |  93.23  |  98.19  |  95.85  |	 Recall@500mm: 1.0000


0.9584612266701336

In [5]:
image_dir_path = "video_viz_campus"
video_output_path = "./test_campus_cam2.avi"
image_list = sorted(os.listdir(image_dir_path), key=lambda x: int(x[6:-4]))
# Read the first image and check its shape to initialize video 
# writer.
read_img = cv2.imread(os.path.join(image_dir_path, image_list[0]))
(height, width, _) = read_img.shape
size = (width, height)
video_writer = cv2.VideoWriter(video_output_path, cv2.VideoWriter_fourcc(*'DIVX'), 15, size)

for image in tqdm(image_list):
    img = cv2.imread(os.path.join(image_dir_path, image))
    video_writer.write(img)
    
video_writer.release()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 222/222 [00:03<00:00, 71.95it/s]


In [ ]:
image_list

['image_0.png',
 'image_1.png',
 'image_2.png',
 'image_3.png',
 'image_4.png',
 'image_5.png',
 'image_6.png',
 'image_7.png',
 'image_8.png',
 'image_9.png',
 'image_10.png',
 'image_11.png',
 'image_12.png',
 'image_13.png',
 'image_14.png',
 'image_15.png',
 'image_16.png',
 'image_17.png',
 'image_18.png',
 'image_19.png',
 'image_20.png',
 'image_21.png',
 'image_22.png',
 'image_23.png',
 'image_24.png',
 'image_25.png',
 'image_26.png',
 'image_27.png',
 'image_28.png',
 'image_29.png',
 'image_30.png',
 'image_31.png',
 'image_32.png',
 'image_33.png',
 'image_34.png',
 'image_35.png',
 'image_36.png',
 'image_37.png',
 'image_38.png',
 'image_39.png',
 'image_40.png',
 'image_41.png',
 'image_42.png',
 'image_43.png',
 'image_44.png',
 'image_45.png',
 'image_46.png',
 'image_47.png',
 'image_48.png',
 'image_49.png',
 'image_50.png',
 'image_51.png',
 'image_52.png',
 'image_53.png',
 'image_54.png',
 'image_55.png',
 'image_56.png',
 'image_57.png',
 'image_58.png',
 'image

In [ ]:
# Well we technically don't need 